In [10]:
def get_library_path()->str:

    cwd = os.getcwd()
    lst = cwd.split(os.sep)[:-1]

    path_lib = f'{os.sep}'

    for elem in lst:
        path_lib = os.path.join(path_lib, elem)

    return path_lib

# imports

import os
import sys

path_to_lib = get_library_path()

sys.path.append(path_to_lib)
import luxgiant_clinical.TwoCatAnalysis as two

import pandas as pd

In [11]:
# load data

folder_path = os.path.join(path_to_lib, 'data/source')

df = pd.read_csv(
    os.path.join(folder_path, 'cleaned_file.csv'), 
    low_memory=False
)

In [12]:
# filter data to keep only patients with disease duration information

mask_patients= (df['Status']=='Patient')
mask_length  = (~df['pdsl'].isnull()) 

df_cases = df[mask_patients & mask_length].reset_index(drop=True)
del df

df_cases["pdsl"] = df_cases["pdsl"].map({"<=5":'PD duration <=5 yrs', ">5":'PD duration >5 yrs'})

df_cases.shape

(7473, 716)

In [13]:
df_on = df_cases[df_cases['hyonoff']=='On'].reset_index(drop=True)

df_on['Stage I-III']= df_on['hystage'].map({'Not severe':1, 'Severe':0})
df_on['Stage IV-V'] = df_on['hystage'].map({'Not severe':0, 'Severe':1})
df_on['UPDRS III-ON']= df_on['updrs_part_iii_total_score'].copy()

variables = ['participant_id', 'pdsl', 'Stage I-III', 'Stage IV-V', 'UPDRS III-ON']

stats_meas1 = {
    'n': ['Stage I-III', 'Stage IV-V'],
    'median' : ['UPDRS III-ON']
}
groups = ['PD duration <=5 yrs', 'PD duration >5 yrs']

# keep only variables for analysis

df_on = df_on[variables].copy()

In [14]:
df_off = df_cases[df_cases['hyonoff']=='Off'].reset_index(drop=True)

df_off['Stage I-III']= df_off['hystage'].map({'Not severe':1, 'Severe':0})
df_off['Stage IV-V'] = df_off['hystage'].map({'Not severe':0, 'Severe':1})
df_off['UPDRS III-OFF']= df_off['updrs_part_iii_total_score'].copy()

variables = ['participant_id', 'pdsl', 'Stage I-III', 'Stage IV-V', 'UPDRS III-OFF']

stats_meas2 = {
    'n': ['Stage I-III', 'Stage IV-V'],
    'median' : ['UPDRS III-OFF']
}

# keep only variables for analysis

df_off = df_off[variables].copy()

In [15]:
summary1 = [
    two.report_proportion(
        data_df    =df_off,
        variables  =stats_meas2['n'],
        groups     =groups,
        grouping_by='pdsl',
        subheader  ='H & Y OFF'
        ),
    two.report_proportion(
        data_df    =df_on,
        variables  =stats_meas1['n'],
        groups     =groups,
        grouping_by='pdsl',
        subheader  ='H & Y ON'
    ),
    two.report_median_iqr(
        data_df    =df_on,
        variables  =stats_meas1['median'],
        groups     =groups,
        grouping_by='pdsl'
    ),
    two.report_median_iqr(
        data_df    =df_off,
        variables  =stats_meas2['median'],
        groups     =groups,
        grouping_by='pdsl'
    )
]

In [16]:
variables_dict = {
    "total_score_for_moca": "MOCA Score",
    "total_score_for_bdi" : "BDI Score",
}
variables_1 = ['participant_id', 'pdsl'] + list(variables_dict.keys())

# statistical measures
stats_meas3 = {
    'mean'  : ["MOCA Score"],
    'median': ["BDI Score"],
}

# keep only variables for analysis

df_cases_1 = df_cases[variables_1].copy()
df_cases_1 = df_cases_1.rename(columns=variables_dict)

In [17]:
summary2 = [ 
    two.report_mean_std(
        data_df    =df_cases_1,
        variables  =stats_meas3['mean'],
        groups     =groups,
        grouping_by='pdsl'
    ),
    two.report_median_iqr(
        data_df    =df_cases_1,
        variables  =stats_meas3['median'],
        groups     =groups,
        grouping_by='pdsl'
    )
]

In [18]:
df_10 = pd.concat(
    summary1 + summary2, axis=0, ignore_index=True
)
df_10 = two.final_formatter(df_10, groups)

df_10.to_csv(os.path.join(path_to_lib, 'data/auxiliar/aux_table_10.csv'), index=False)
df_10

,Variable,Statistical Measure,PD duration <=5 yrs,PD duration >5 yrs,p-value,Total,Available Samples for Analysis
0,H & Y OFF,,,,,,
1,Stage I-III,n (%),1465 (97.7),885 (88.4),p<0.001,2350 (94.0),2500
2,Stage IV-V,n (%),34 (2.3),116 (11.6),p<0.001,150 (6.0),2500
3,H & Y ON,,,,,,
4,Stage I-III,n (%),1923 (98.1),1738 (93.1),p<0.001,3661 (95.7),3827
5,Stage IV-V,n (%),38 (1.9),128 (6.9),p<0.001,166 (4.3),3827
6,UPDRS III-ON,median (IQR),18.0 (11.0 - 27.0),23.0 (15.0 - 32.0),p<0.001,21.0 (13.0 - 30.0),3168
7,UPDRS III-OFF,median (IQR),36.0 (25.0 - 45.0),41.0 (30.0 - 49.0),p<0.001,37.0 (27.0 - 46.0),1547
8,MOCA Score,mean (SD),24.0 (5.5),23.4 (5.5),p<0.001,23.7 (5.5),5149
9,BDI Score,median (IQR),8.0 (5.0 - 12.0),10.0 (6.0 - 15.0),p<0.001,9.0 (5.0 - 13.0),5701
